<a href="https://colab.research.google.com/github/tommyEzreal/study_low_level/blob/main/pytorch/%EB%AA%A8%EB%91%90%EB%A5%BC%EC%9C%84%ED%95%9C%EB%94%A5%EB%9F%AC%EB%8B%9D2/Weight_initialization%26Dropout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
# ReLU 
# f(x) = max(0,x)
# torch.nn.relu(x)


In [3]:
# Restricted Boltzman Machine(RBM)

# restricted: no connection within a layer 
# KL-divergence : compare actual to recreation 

# x -> y encoding  y -> x' decoding 

# DeepBeliefNetwork's pretrain -> finetuning 구조 

In [ ]:
# Xavier initialization 
# He initialization 

# dependent on layer's feature 

# normal & uniform distribution initialization 


In [4]:
# torch.nn xavier_uniform_ 
def xavier_uniform_(tensor, gain=1):
  fan_in, fan_out = _calculate_fan_in_and_fan_out(tensor)
  std = gain * math.sqrt(2.0 / (fan_in + fan_out))
  a = math.sqrt(3.0) * std
  with torch.no_grad():
    return tensor.uniform(-a,a)

In [ ]:
linear1 = torch.nn.Linear(784,256, bias=True)
linear2 = torch.nn.Linear(256,256, bias=True)
linear3 = torch.nn.Linear(256,10, bias=True)
relu = torch.nn.ReLU()

# weight initialize with xaiver_uniform 
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Mnist with xavier initalization 

In [5]:
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random 

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [7]:
learning_rate = 0.001
epochs = 15
batch_size = 64

In [8]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw



In [9]:
# dataloader
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size = batch_size,
                                          shuffle=True,
                                          drop_last=True)


In [10]:
linear1 = torch.nn.Linear(784,256, bias=True)
linear2 = torch.nn.Linear(256,256, bias=True)
linear3 = torch.nn.Linear(256,10, bias=True)
relu = torch.nn.ReLU()

# weight initialize with xaiver_uniform 
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[-0.0215, -0.0894,  0.0598,  ...,  0.0200,  0.0203,  0.1212],
        [ 0.0078,  0.1378,  0.0920,  ...,  0.0975,  0.1458, -0.0302],
        [ 0.1270, -0.1296,  0.1049,  ...,  0.0124,  0.1173, -0.0901],
        ...,
        [ 0.0661, -0.1025,  0.1437,  ...,  0.0784,  0.0977, -0.0396],
        [ 0.0430, -0.1274, -0.0134,  ..., -0.0582,  0.1201,  0.1479],
        [-0.1433,  0.0200, -0.0568,  ...,  0.0787,  0.0428, -0.0036]],
       requires_grad=True)

In [11]:
# model 
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)


In [14]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [16]:
total_batch = len(data_loader)
for epoch in range(epochs+1):
  avg_cost = 0

  for X,Y in data_loader:
    X = X.view(-1, 784).to(device)
    Y = Y.to(device)

    h = model(X)
    cost = criterion(h,Y)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    avg_cost += cost / total_batch

  print('Epoch: {}/{} cost:{:.4f}'.format(
      epoch, epochs, avg_cost
  ))
    

Epoch: 0/15 cost:0.2238
Epoch: 1/15 cost:0.0865
Epoch: 2/15 cost:0.0571
Epoch: 3/15 cost:0.0430
Epoch: 4/15 cost:0.0342
Epoch: 5/15 cost:0.0251
Epoch: 6/15 cost:0.0220
Epoch: 7/15 cost:0.0213
Epoch: 8/15 cost:0.0149
Epoch: 9/15 cost:0.0162
Epoch: 10/15 cost:0.0148
Epoch: 11/15 cost:0.0110
Epoch: 12/15 cost:0.0115
Epoch: 13/15 cost:0.0117
Epoch: 14/15 cost:0.0103
Epoch: 15/15 cost:0.0086


In [17]:
# Test 

# Test the model using test sets
with torch.no_grad():
    X_test = mnist_test.test_data.view(-1, 28 * 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

    # Get one and predict
    r = random.randint(0, len(mnist_test) - 1)
    X_single_data = mnist_test.test_data[r:r + 1].view(-1, 28 * 28).float().to(device)
    Y_single_data = mnist_test.test_labels[r:r + 1].to(device)

    print('Label: ', Y_single_data.item())
    single_prediction = model(X_single_data)
    print('Prediction: ', torch.argmax(single_prediction, 1).item())

Accuracy: 0.9822999835014343
Label:  8
Prediction:  8


/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.8/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Dropout
- randomly off nodes in layers

In [22]:
drop_prob = 0.1 
dropout = torch.nn.Dropout(p=drop_prob)

linear1 = torch.nn.Linear(784,256, bias=True)
model = torch.nn.Sequential(linear1, relu, dropout)

In [28]:
# evaluation, test set - dropout=False 

model.train() # 신경망의 학습모드 전환 
model.eval() # 신경망의 추론모드 전환 

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.1, inplace=False)
)